In [1]:
using CSV
using DataFrames
using Serialization

In [2]:
metadata = DataFrame(CSV.File("../data/hospital_meta.csv"));
filter!(row -> row.in_forecast, metadata);

In [3]:
states = [("Maryland", "MD")]
println(states)

[("Maryland", "MD")]


In [4]:
function filter_regions(regions)
    ids = [r.id for r in regions]
    regions = [r for r in unique(regions) if sum(ids .== r.id) > 1]
    return unique(regions)
end;

In [5]:
hsas = collect(zip(metadata.hsa_name, metadata.hsa_id))
hsas = filter(s -> !(ismissing(s[1]) || ismissing(s[2])), hsas)
hsas = map(s -> (name = s[1], id = string(s[2])), hsas)
hsas = filter_regions(hsas)
hsas = sort(hsas)

4-element Array{NamedTuple{(:name, :id),Tuple{String,String}},1}:
 (name = "Baltimore, MD", id = "21002")
 (name = "Easton, MD", id = "21011")
 (name = "Rockville, MD", id = "21028")
 (name = "Salisbury, MD", id = "21029")

In [6]:
hrrs = collect(zip(metadata.hrr_name, metadata.hrr_id))
hrrs = filter(s -> !(ismissing(s[1]) || ismissing(s[2])), hrrs)
hrrs = map(s -> (name = s[1], id = string(s[2])), hrrs)
hrrs = filter_regions(hrrs)
hrrs = sort(hrrs)

4-element Array{NamedTuple{(:name, :id),Tuple{String,String}},1}:
 (name = "Baltimore, MD", id = "223")
 (name = "Salisbury, MD", id = "225")
 (name = "Takoma Park, MD", id = "226")
 (name = "Washington, DC", id = "113")

In [7]:
# city_id(c) = c |> lowercase |> x -> replace(x, " " => "_");
city_id(c) = c;

In [8]:
cities = map(row -> (row.city, city_id(row.city)), eachrow(metadata))
cities = filter(s -> !(ismissing(s[1]) || ismissing(s[2])), cities)
cities = map(s -> (name = s[1], id = string(s[2])), cities)
cities = filter_regions(cities)
cities = sort(cities)

3-element Array{NamedTuple{(:name, :id),Tuple{String,String}},1}:
 (name = "Baltimore", id = "Baltimore")
 (name = "Germantown", id = "Germantown")
 (name = "Towson", id = "Towson")

In [9]:
# county_id(c) = split(c, " County")[1] |> lowercase |> x -> replace(x, " " => "_");
county_id(c) = c;

In [10]:
counties = map(row -> (row.county, county_id(row.county)), eachrow(metadata))
counties = filter(s -> !(ismissing(s[1]) || ismissing(s[2])), counties)
counties = map(s -> (name = s[1], id = string(s[2])), counties)
counties = filter_regions(counties)
counties = sort(counties)

6-element Array{NamedTuple{(:name, :id),Tuple{String,String}},1}:
 (name = "Anne Arundel County", id = "Anne Arundel County")
 (name = "Baltimore City", id = "Baltimore City")
 (name = "Baltimore County", id = "Baltimore County")
 (name = "Harford County", id = "Harford County")
 (name = "Montgomery County", id = "Montgomery County")
 (name = "Prince Georges County", id = "Prince Georges County")

In [11]:
# mdregion_id(c) = c[8:end];
mdregion_id(c) = c;

In [12]:
mdregions = map(row -> (row.healthcare_region, mdregion_id(row.healthcare_region)), eachrow(metadata))
mdregions = filter(s -> !(ismissing(s[1]) || ismissing(s[2])), mdregions)
mdregions = map(s -> (name = s[1], id = string(s[2])), mdregions)
mdregions = unique(mdregions)
mdregions = sort(mdregions)

5-element Array{NamedTuple{(:name, :id),Tuple{String,String}},1}:
 (name = "Region I", id = "Region I")
 (name = "Region II", id = "Region II")
 (name = "Region III", id = "Region III")
 (name = "Region IV", id = "Region IV")
 (name = "Region V", id = "Region V")

In [13]:
state_regions = [(region_type = :state, region_name = s[1], region_id = s[2]) for s in states];

In [14]:
system_regions = [];

In [15]:
hsa_regions = [(region_type = :hsa, region_name = s.name, region_id = s.id) for s in hsas];
hrr_regions = [(region_type = :hrr, region_name = s.name, region_id = s.id) for s in hrrs];

In [16]:
city_regions   = [(region_type = :city,   region_name = s[1], region_id = s[2]) for s in cities];
county_regions = [(region_type = :county, region_name = s[1], region_id = s[2]) for s in counties];

In [17]:
health_regions = [(region_type = :healthcareregion, region_name = s[1], region_id = s[2]) for s in mdregions];

In [18]:
regions = vcat(state_regions, system_regions, hsa_regions, hrr_regions, city_regions, county_regions, health_regions);

In [19]:
serialize("../data/regions.jlser", regions)